In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

In [2]:
# Define a dictionary to store all the datasets 
season_statistics = {} 

for year in range(1994, 2024): 
    # Import the CSV file for the corresponding year as a pandas df 
    df = pd.read_csv(f"./data/{year}.csv") 
    
    # Add the dataframe to our dictionary of dataframes 
    season_statistics[year] = df

In [3]:
# Concatenate all season data
df = pd.concat(season_statistics.values(), ignore_index=True)

# Define Categorical and Numerical Columns for Pre-Processing
selected_features = df.columns.drop(['Champion', 'Arena', 'Team', 'Year', 'G'])
# selected_features = ['W%', 'FG%', 'FGA', '3P%', 'FT%', 'ORB', 'DRB', 'AST', 'STL', 'BLK', 
#                      'PPG', 'OPPG', 'Age', 'ORtg', 'DRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'eFG%', 'Attend./G']

# Create a Data PreProcessor
preprocessor = ColumnTransformer(transformers = [('num', StandardScaler(), selected_features)])

# Separate the Predictors and Response
X = df[selected_features]
y = df['Champion'].values

# Pre-Process the Input Data
X = preprocessor.fit_transform(X)

In [4]:
# Define the K-Fold Cross Validation Sets
kf = KFold(n_splits = 5, shuffle = True, random_state = 669)

# Create a List for each Model and its Accuracy
model_history, mse_history = [], []

# Perform K-Fold Cross Validation
for train_index, test_index in kf.split(X):
    
    # Split into Training and Testing Datasets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Build the Neural Network Model
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(128, activation='relu'),
        Dense(128, activation='relu'),
        Dense(128, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)
    
    # Predict and evaluate
    y_pred = model.predict(X_test).flatten()
    y_pred, y_test = (y_pred * 100), (y_test * 100)
    mse = mean_squared_error(y_test, y_pred)
    
    # Append the Model and MSE to the respective lists
    model_history.append(model)
    mse_history.append(mse)

Epoch 1/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0517 - mse: 0.0517
Epoch 2/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0244 - mse: 0.0244
Epoch 3/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0154 - mse: 0.0154
Epoch 4/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0123 - mse: 0.0123
Epoch 5/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0089 - mse: 0.0089
Epoch 6/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0071 - mse: 0.0071
Epoch 7/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0052 - mse: 0.0052
Epoch 8/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0043 - mse: 0.0043
Epoch 9/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0036 - mse: 0.0036
Epoch 10/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0047 - mse: 0.0047
Epoch 11/100
23/23 [==============================] - 0s 1m

23/23 [==============================] - 0s 1ms/step - loss: 8.7028e-04 - mse: 8.7028e-04
Epoch 84/100
23/23 [==============================] - 0s 1ms/step - loss: 6.6985e-04 - mse: 6.6985e-04
Epoch 85/100
23/23 [==============================] - 0s 1ms/step - loss: 5.8028e-04 - mse: 5.8028e-04
Epoch 86/100
23/23 [==============================] - 0s 1ms/step - loss: 6.1959e-04 - mse: 6.1959e-04
Epoch 87/100
23/23 [==============================] - 0s 1ms/step - loss: 4.8188e-04 - mse: 4.8188e-04
Epoch 88/100
23/23 [==============================] - 0s 1ms/step - loss: 5.0677e-04 - mse: 5.0677e-04
Epoch 89/100
23/23 [==============================] - 0s 1ms/step - loss: 4.4102e-04 - mse: 4.4102e-04
Epoch 90/100
23/23 [==============================] - 0s 1ms/step - loss: 6.3295e-04 - mse: 6.3295e-04
Epoch 91/100
23/23 [==============================] - 0s 1ms/step - loss: 7.4624e-04 - mse: 7.4624e-04
Epoch 92/100
23/23 [==============================] - 0s 1ms/step - loss: 4.7746e-04 -

23/23 [==============================] - 0s 1ms/step - loss: 5.7713e-04 - mse: 5.7713e-04
Epoch 65/100
23/23 [==============================] - 0s 1ms/step - loss: 3.7363e-04 - mse: 3.7363e-04
Epoch 66/100
23/23 [==============================] - 0s 1ms/step - loss: 3.6357e-04 - mse: 3.6357e-04
Epoch 67/100
23/23 [==============================] - 0s 1ms/step - loss: 3.4203e-04 - mse: 3.4203e-04
Epoch 68/100
23/23 [==============================] - 0s 1ms/step - loss: 3.4579e-04 - mse: 3.4579e-04
Epoch 69/100
23/23 [==============================] - 0s 1ms/step - loss: 3.0733e-04 - mse: 3.0733e-04
Epoch 70/100
23/23 [==============================] - 0s 1ms/step - loss: 2.8768e-04 - mse: 2.8768e-04
Epoch 71/100
23/23 [==============================] - 0s 1ms/step - loss: 3.3405e-04 - mse: 3.3405e-04
Epoch 72/100
23/23 [==============================] - 0s 1ms/step - loss: 2.6941e-04 - mse: 2.6941e-04
Epoch 73/100
23/23 [==============================] - 0s 1ms/step - loss: 1.7716e-04 -

23/23 [==============================] - 0s 1ms/step - loss: 0.0031 - mse: 0.0031
Epoch 47/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0038 - mse: 0.0038
Epoch 48/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0029 - mse: 0.0029    
Epoch 49/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0027 - mse: 0.0027    
Epoch 50/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0021 - mse: 0.0021
Epoch 51/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0013 - mse: 0.0013
Epoch 52/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0037 - mse: 0.0037    
Epoch 53/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0027 - mse: 0.0027
Epoch 54/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0022 - mse: 0.0022
Epoch 55/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0012 - mse: 0.0012
Epoch 56/100
23/23 [==============================]

23/23 [==============================] - 0s 1ms/step - loss: 0.0034 - mse: 0.0034
Epoch 29/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0052 - mse: 0.0052
Epoch 30/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0050 - mse: 0.0050
Epoch 31/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0047 - mse: 0.0047
Epoch 32/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0037 - mse: 0.0037    
Epoch 33/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0022 - mse: 0.0022
Epoch 34/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0013 - mse: 0.0013    
Epoch 35/100
23/23 [==============================] - 0s 1ms/step - loss: 6.6410e-04 - mse: 6.6410e-04
Epoch 36/100
23/23 [==============================] - 0s 1ms/step - loss: 4.9769e-04 - mse: 4.9769e-04
Epoch 37/100
23/23 [==============================] - 0s 1ms/step - loss: 7.5304e-04 - mse: 7.5304e-04
Epoch 38/100
23/23 [===========

23/23 [==============================] - 0s 1ms/step - loss: 0.0077 - mse: 0.0077
Epoch 10/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0049 - mse: 0.0049
Epoch 11/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0030 - mse: 0.0030
Epoch 12/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0035 - mse: 0.0035
Epoch 13/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0028 - mse: 0.0028
Epoch 14/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0028 - mse: 0.0028
Epoch 15/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0012 - mse: 0.0012
Epoch 16/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0014 - mse: 0.0014
Epoch 17/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0013 - mse: 0.0013    
Epoch 18/100
23/23 [==============================] - 0s 1ms/step - loss: 0.0013 - mse: 0.0013
Epoch 19/100
23/23 [==============================] - 0s 1m

23/23 [==============================] - 0s 1ms/step - loss: 1.0599e-04 - mse: 1.0599e-04
Epoch 92/100
23/23 [==============================] - 0s 1ms/step - loss: 6.2148e-05 - mse: 6.2148e-05
Epoch 93/100
23/23 [==============================] - 0s 1ms/step - loss: 1.6169e-04 - mse: 1.6169e-04
Epoch 94/100
23/23 [==============================] - 0s 1ms/step - loss: 2.3563e-04 - mse: 2.3563e-04
Epoch 95/100
23/23 [==============================] - 0s 1ms/step - loss: 1.6946e-04 - mse: 1.6946e-04
Epoch 96/100
23/23 [==============================] - 0s 1ms/step - loss: 2.8552e-04 - mse: 2.8552e-04
Epoch 97/100
23/23 [==============================] - 0s 1ms/step - loss: 2.7402e-04 - mse: 2.7402e-04
Epoch 98/100
23/23 [==============================] - 0s 1ms/step - loss: 2.5780e-04 - mse: 2.5780e-04
Epoch 99/100
23/23 [==============================] - 0s 1ms/step - loss: 1.9435e-04 - mse: 1.9435e-04
Epoch 100/100
6/6 [==============================] - 0s 734us/step


In [5]:
# Print Performance Metrics
print("Average MSE:", np.mean(mse_history), '\n')
for i in range(len(model_history)):
    print(f"Model #{i}, MSE: {mse_history[i]}")
    
# Get the Best Model
idx = mse_history.index(min(mse_history))
best_model = model_history[idx]

# Perform Accuracy Test
accuracy = 0
print("\n\nChampionship Prediction (Historic)\n----------------------------------\n")
for year, df in season_statistics.items():
    
    # Get the Actual Champion
    real_champion = df[df['Champion'] == 1]['Team'].values[0]
    
    # Get the Model Prediction
    X = preprocessor.fit_transform(df[selected_features])
    y = best_model.predict(X)
    pred_champion = df.iloc[np.argmax(y)]['Team']
    
    # Evaluate Results
    accuracy += int(real_champion == pred_champion)
    print(f"{year} NBA Season")
    print(f"\tActual Champion: {real_champion}")
    print(f"\tPredicted Champion: {pred_champion}")
    print()
    
print(f"Model Accuracy: {(accuracy / len(season_statistics) * 100):.2f}% ({accuracy}/{len(season_statistics)})")

Average MSE: 281.89280386403027 

Model #0, MSE: 295.764775376108
Model #1, MSE: 273.85340820120865
Model #2, MSE: 362.18161323830327
Model #3, MSE: 238.01442380002467
Model #4, MSE: 239.64979870450694


Championship Prediction (Historic)
----------------------------------

1/1 [==============================] - 0s 14ms/step
1994 NBA Season
	Actual Champion: Houston Rockets
	Predicted Champion: Houston Rockets

1/1 [==============================] - 0s 14ms/step
1995 NBA Season
	Actual Champion: Houston Rockets
	Predicted Champion: Orlando Magic

1/1 [==============================] - 0s 12ms/step
1996 NBA Season
	Actual Champion: Chicago Bulls
	Predicted Champion: Chicago Bulls

1/1 [==============================] - 0s 12ms/step
1997 NBA Season
	Actual Champion: Chicago Bulls
	Predicted Champion: Chicago Bulls

1/1 [==============================] - 0s 14ms/step
1998 NBA Season
	Actual Champion: Chicago Bulls
	Predicted Champion: Chicago Bulls

1/1 [==============================] - 

In [6]:
# Get the 2024 NBA Season Data
df = pd.read_csv(f"./data/2024.csv")

# Get the Model Prediction
X = preprocessor.fit_transform(df[selected_features])
y = best_model.predict(X)
pred_champion = df.iloc[np.argmax(y)]['Team']

# Evaluate Results
print(f"2024 Predicted Champion: {pred_champion}")

# Print Championship Probabilities Per Team
print("\n\nChampionship Probabilities (By Team)\n-------------------------------------")
for i in range(len(y)):
    print(f"{df.iloc[i]['Team']}: {(y[i][0] * 100):.2f}%")

1/1 [==============================] - 0s 12ms/step
2024 Predicted Champion: Boston Celtics


Championship Probabilities (By Team)
-------------------------------------
Atlanta Hawks: 1.32%
Boston Celtics: 50.76%
Brooklyn Nets: 0.19%
Charlotte Hornets: -0.71%
Chicago Bulls: 0.33%
Cleveland Cavaliers: 18.90%
Dallas Mavericks: 4.05%
Denver Nuggets: 19.59%
Detroit Pistons: -0.73%
Golden State Warriors: 2.42%
Houston Rockets: 1.52%
Indiana Pacers: 2.52%
Los Angeles Clippers: 15.12%
Los Angeles Lakers: 3.37%
Memphis Grizzlies: 2.17%
Miami Heat: 2.09%
Milwaukee Bucks: 9.27%
Minnesota Timberwolves: 49.97%
New Orleans Pelicans: 8.27%
New York Knicks: 11.44%
Oklahoma City Thunder: 27.96%
Orlando Magic: 16.73%
Philadelphia 76ers: 0.23%
Phoenix Suns: 21.91%
Portland Trail Blazers: -0.23%
Sacramento Kings: 5.49%
San Antonio Spurs: -0.12%
Toronto Raptors: 3.41%
Utah Jazz: -1.12%
Washington Wizards: 3.25%
